# Random Walk Metrics


## Modules

In [77]:
import numpy as np
import pandas as pd
import math

from scipy.spatial import distance

from scipy.stats  import wrapcauchy
from scipy.stats import levy_stable

import plotly.graph_objects as go


## Class

In [78]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
    and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Activity 1 - Path length - (BM1 vs BM2 vs CRW)
    - Write a function that returns a Brownian Motion (BM) trajectory in pandas df.
    - Write a function that returns a Correlated Random Walk (CRW) trajectory in pandas df.
    - Write a function that returns the path length for a given trajectory.
    - Compare at least the path length of three trajectories as shown in the figure below.
    - Display the results using plotly.

In [79]:
# Function to get Brownian Motion trajectory, using random 90 degree rotation angle directions
def bm_traj(speed=3, n_steps=1000, s_pos=[0,0]):
    trajectory_bw = pd.DataFrame(columns=['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

    trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

    velocity = Vec2d(speed, 0) # Starting position for velocity vector

    for i in range(n_steps):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        
        velocity = velocity.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{'x_pos':trajectory_bw.x_pos[i] + velocity.x, 'y_pos':trajectory_bw.y_pos[i] + velocity.y}])
        trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

    return trajectory_bw
    

In [80]:
# Function to get correlated random walk trajectory using Wrapped Cauchy Distribution for turning angles on each step/iteration
def crw_traj(speed=5, coefficient=0.4, n_steps=1000, s_pos=[0,0]):
    
    r = wrapcauchy.rvs(loc=0, c=coefficient, size=n_steps)

    velocity = Vec2d(speed, 0) # Velocity vector in starting position

    trajectory_crw = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])

    trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True) 

    for i in range(n_steps):
        turn_angle = r[i]
        
        velocity = velocity.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{'x_pos':trajectory_crw.x_pos[i] + velocity.x, 'y_pos':trajectory_crw.y_pos[i] + velocity.y}])
        trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True)
        
    return trajectory_crw
    

In [81]:
# Function to get the path length of a given dataframe trajectory
def pl_traj(df, s_pos=[0]):
    pl = pd.DataFrame(columns=['y_pos'])
    temp_df = pd.DataFrame([{'y_pos':s_pos[0]}])

    pl = pd.concat([pl, temp_df], ignore_index=True)

    for i in range(len(df)-1): # minus one because len counts the dataframe header
        temp_df = pd.DataFrame([{'y_pos':pl.y_pos[i] + distance.euclidean(df.iloc[i], df.iloc[i+1])}])
        pl = pd.concat([pl, temp_df], ignore_index=True)
        
    return pl


In [82]:
# Obtaining the correspondent trajectories
bm_tr_3 = bm_traj(3)
bm_tr_6 = bm_traj(6)
crw_tr_6 = crw_traj(6, coefficient=0.5)

# Obtaining path length dataframe for each trajectory
bm_pl_3 = pl_traj(bm_tr_3)
bm_pl_6 = pl_traj(bm_tr_6)
crw_pl_6 = pl_traj(crw_tr_6)

In [83]:
# Plotting of three path length trajectories for both Brownian Motion and Correlated Random Walks
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= bm_pl_3.index,
    y= bm_pl_3.y_pos,
    name='BM - 3',
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= bm_pl_6.index,
    y= bm_pl_6.y_pos,
    name='BM - 6',
    marker = dict(size=2),
    line = dict(width=6),
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= crw_pl_6.index,
    y= crw_pl_6.y_pos,
    name='CRW 0.5 - 6',
    mode='lines',
    showlegend=True
))

fig.update_layout(
    title='Path Lengths',
    height=800,
    xaxis_title="times",
    yaxis_title="distnace",
)

fig.show()
    